In [1]:
%load_ext lab_black
%cd ../../..

/mnt/h/hev/log-analytics


In [14]:
import torch
import numpy as np
import pandas as pd
import pickle

In [15]:
df = pd.read_csv("data/ori/test.csv")

In [3]:
deck1 = torch.load("results/distilbert-base-uncased-ver8/distilbert-base-uncased-focal-AdamW-lr1e-05-ver8-os10_4-deck1.pth")
deck2 = torch.load("results/distilbert-base-uncased-ver8/distilbert-base-uncased-focal-AdamW-lr1e-05-ver8-os10_4-deck2.pth")

In [4]:
deck1["feat"].shape, deck2["feat"].shape

(torch.Size([421079, 768]), torch.Size([1095951, 768]))

In [5]:
deck1["feat_"] = deck1["feat"].cuda()
deck2["feat_"] = deck2["feat"].cuda()

In [6]:
j = 0
shown = False

In [7]:
dist_ = torch.norm(deck1["feat_"] - deck2["feat_"][j, None], p=None, dim=1)
values_, indices_ = dist_.topk(4, largest=False)
tlevels = deck1["tlevel"][indices_]
print("values:", values_)
print("indices:", indices_)
print("id:", deck1["tlevel"][indices_])
j += 1

values: tensor([0.0600, 0.0960, 0.0980, 0.1035], device='cuda:0')
indices: tensor([250795,  39129, 173781, 345343], device='cuda:0')
id: tensor([0, 0, 0, 0])


In [21]:
while shown or (deck1["tlevel"][indices_[0]] in [0, 1, 3, 5] and values_[0] < 0.5):
    # while shown or (deck1["tlevel"][indices_[0]] in [0, 1, 3, 5]):
    shown = False

    dist_ = torch.norm(deck1["feat_"] - deck2["feat_"][j, None], p=None, dim=1)
    values_, indices_ = dist_.topk(4, largest=False)
    tlevels = deck1["tlevel"][indices_]
    # print("values:", values)
    # print("indices:", indices)
    # print("id:", deck1["tlevel"][indices])
    j += 1

In [22]:
print("*  Submit :", df.full_log[deck2["ids"][j - 1][0] - 1000000])
print("----------------------------------------------------------------------------------------")
print("*   index :", j - 1)
print("*    dist :", " ".join(map(lambda x: f"{x:.4f}", values_.cpu().tolist())))
print("* fc-level:", deck2["fclevel"][j - 1])
print("----------------------------------------------------------------------------------------")

for idx in indices_:
    print("* Level", deck1["tlevel"][idx].item(), ":", deck1["otext"][idx])
    print("----------------------------------------------------------------------------------------")

shown = True

*  Submit : System Audit: CIS - RHEL7 - 4.1.2 - Network parameters - IP send redirects enabled {CIS: 4.1.2 RHEL7} {PCI_DSS: 2.2.4}. File: /proc/sys/net/ipv4/conf/all/send_redirects. Reference: https://benchmarks.cisecurity.org/tools2/linux/CIS_Red_Hat_Enterprise_Linux_7_Benchmark_v1.1.0.pdf .
----------------------------------------------------------------------------------------
*   index : 13638
*    dist : 0.5428 0.6414 0.8143 0.8165
* fc-level: tensor(1)
----------------------------------------------------------------------------------------
* Level 1 : System Audit: CIS - RHEL7 - 4.1.2 - Network parameters - IP send redirects enabled {CIS: 4.1.2 RHEL7} {PCI_DSS: 2.2.4}. File: /proc/sys/net/ipv4/conf/all/send_redirects. Reference: https://bench month ks.cisecurity.org/tools2/linux/CIS_Red_Hat_Enterprise_Linux_7_Bench month k_v1.1.0.pdf
----------------------------------------------------------------------------------------
* Level 1 : System Audit: CIS - RHEL7 - 4.1. - Network para

---

4개가 다 일치할 때는 level별로 threshold를 따로 두는 방식으로 하고
* level0 - 좀 낮게 (비교군이 많기 때문에) - 0.3 ? 평균 dist가 0.3?
* level1 - 길이는 길지만 거의 비슷한 값들인듯? 대부분 0.01을 넘지 않음 - 0.2 or 0.1?
* level2 - 
* level3 - 꽤 높게 (변화무쌍하니까) - 1.0?
* level4 - 
* level5 - 꽤 높게 (변화무쌍하니까) - 1.0
* level6 - 나름 변화가 큼 - 0.8
* 섞인 경우 - 비슷한 케이스가 별로 없었다는 뜻이니까 낮게 - 0.2

2, 4, 6에 대해서는 아직 데이터가 더 모여야 하겠다

level0 길이가 긴거는 맞아도 dist가 높게(0.3이상) 나오고, 길이가 짧은건 틀려도 dist가 탖게 나오는 것 같은데...  
길이도 입력 인수로 해야할까?

※ TODO: 전체에 대해 dist 구한 다음에 각 level/군집별로 top dist 분포가 어떻게 나타나는지 그래프를 그려보자